In [1]:
import requests
import json
import sqlite3
import pandas as pd
from datetime import date, timedelta

In [2]:
conn = sqlite3.connect('war.db')
cursor = conn.cursor()

In [3]:
# Отримуємо підключення до бази даних
conn = sqlite3.connect('war.db')
cursor = conn.cursor()

In [5]:
# Створюємо таблицю russian_losses
cursor.execute('''
    create table russian_losses(
    day,
    date,
    personnel_units,
    tanks,
    armoured_fighting_vehicles, 
    artillery_systems,
    mlrs,
    aa_warfare_systems,
    planes,
    helicopters,
    vehicles_fuel_tanks,
    warships_cutters,
    cruise_missiles,
    uav_systems,
    special_military_equip,
    atgm_srbm_systems
    )
''')

З АРІ отримуємо загальні втрати росіян станом на кожен день, в базу даних будуть записуватися втрати які були здійснені в цей 
день. В API є один маленький недолік немає даних за 26.02.2022, тому дані за 25 та 27 лютого будуть отримані за межами 
циклу. В циклі while обсчислються втрати росіян за конкретний день шляхом віднімання від даних отриманих з API за цей день даних
за попередній день

In [10]:
# Перевіряємо чи є дані за за 25.02.2022 і автоматично перевіряємо дані за 27.02.23
cursor.execute("SELECT * FROM russian_losses WHERE date=?", ('2022-02-25',))
existing_data = cursor.fetchone()
    
# Якщо дані відсутні, тоді отримуємо їх з API
if not existing_data:
    req = requests.get(f'https://russianwarship.rip/api/v1/statistics/2022-02-25')
    data = req.text
    json_str = data
    data_dict = json.loads(json_str)
    
    # Вставляємо дані в таблицю russian_losses
    cursor.execute('''INSERT INTO russian_losses (day, date, personnel_units, tanks, armoured_fighting_vehicles, 
                    artillery_systems, mlrs, aa_warfare_systems, planes, helicopters, vehicles_fuel_tanks,
                    warships_cutters, cruise_missiles, uav_systems, special_military_equip, atgm_srbm_systems)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                    (data_dict['data']['day'], data_dict['data']['date'], data_dict['data']['stats']['personnel_units'],
                    data_dict['data']['stats']['tanks'], data_dict['data']['stats']['armoured_fighting_vehicles'],
                    data_dict['data']['stats']['artillery_systems'], data_dict['data']['stats']['mlrs'],
                    data_dict['data']['stats']['aa_warfare_systems'], data_dict['data']['stats']['planes'],
                    data_dict['data']['stats']['helicopters'], data_dict['data']['stats']['vehicles_fuel_tanks'],
                    data_dict['data']['stats']['warships_cutters'], data_dict['data']['stats']['cruise_missiles'],
                    data_dict['data']['stats']['uav_systems'], data_dict['data']['stats']['special_military_equip'],
                    data_dict['data']['stats']['atgm_srbm_systems']))
    # присвоюємо дані про втрати змінним
    prev_personnel_units = data_dict['data']['stats']['personnel_units']
    prev_tanks = data_dict['data']['stats']['tanks']
    prev_armoured_fighting_vehicles = data_dict['data']['stats']['armoured_fighting_vehicles']
    prev_artillery_systems = data_dict['data']['stats']['artillery_systems']
    prev_mlrs = data_dict['data']['stats']['mlrs']
    prev_aa_warfare_systems = data_dict['data']['stats']['aa_warfare_systems']
    prev_planes = data_dict['data']['stats']['planes']
    prev_helicopters = data_dict['data']['stats']['helicopters']
    prev_vehicles_fuel_tanks = data_dict['data']['stats']['vehicles_fuel_tanks'] 
    prev_warships_cutters = data_dict['data']['stats']['warships_cutters']
    prev_cruise_missiles = data_dict['data']['stats']['cruise_missiles']
    prev_uav_systems = data_dict['data']['stats']['uav_systems']
    prev_special_military_equip = data_dict['data']['stats']['special_military_equip']
    prev_atgm_srbm_systems = data_dict['data']['stats']['atgm_srbm_systems']

    # отримуємо дані за 27.02.2023
    req = requests.get(f'https://russianwarship.rip/api/v1/statistics/2022-02-27')
    data = req.text
    json_str = data
    data_dict = json.loads(json_str)
    
    # Вставляємо дані в таблицю russian_losses віднімаючи від даних за 27.02.2022 дані за 25.02.2022
    cursor.execute('''INSERT INTO russian_losses (day, date, personnel_units, tanks, armoured_fighting_vehicles, 
                    artillery_systems, mlrs, aa_warfare_systems, planes, helicopters, vehicles_fuel_tanks,
                    warships_cutters, cruise_missiles, uav_systems, special_military_equip, atgm_srbm_systems)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                    (data_dict['data']['day'], data_dict['data']['date'],
                    data_dict['data']['stats']['personnel_units'] - prev_personnel_units,
                    data_dict['data']['stats']['tanks'] - prev_tanks,
                    data_dict['data']['stats']['armoured_fighting_vehicles'] - prev_armoured_fighting_vehicles,
                    data_dict['data']['stats']['artillery_systems'] - prev_artillery_systems,
                    data_dict['data']['stats']['mlrs'] - prev_mlrs,
                    data_dict['data']['stats']['aa_warfare_systems'] - prev_aa_warfare_systems, 
                    data_dict['data']['stats']['planes'] - prev_planes,
                    data_dict['data']['stats']['helicopters'] - prev_helicopters,
                    data_dict['data']['stats']['vehicles_fuel_tanks'] - prev_vehicles_fuel_tanks,
                    data_dict['data']['stats']['warships_cutters'] - prev_warships_cutters,
                    data_dict['data']['stats']['cruise_missiles'] - prev_cruise_missiles,
                    data_dict['data']['stats']['uav_systems'] - prev_uav_systems,
                    data_dict['data']['stats']['special_military_equip'] - prev_special_military_equip,
                    data_dict['data']['stats']['atgm_srbm_systems'] - prev_atgm_srbm_systems))
    # Зберігаємо зміни
    conn.commit()

In [4]:
# Дати для циклу
start_date = date(2022, 2, 27)
end_date = date.today()
    
# Пройдемося по датам від '2022-02-27' до сьогоднішньої дати
current_date = start_date
delta = timedelta(days=1)

# Заходимо в цикл та знаходиом втрати росіян за кожен день
while current_date <= end_date:
    # Отримуємо дату у форматі "yyyy-mm-dd" з поточної дати
    date_str = current_date.isoformat()
        
    # Отримуємо дані з API для поточної дати та перевіряємо чи є ці дані в таблиці
    cursor.execute("SELECT * FROM russian_losses WHERE date=?", (date_str,))
    existing_data = cursor.fetchone()
    
    # Якщо дані відсутні, тоді отримуємо їх з API
    if not existing_data:
        req = requests.get(f'https://russianwarship.rip/api/v1/statistics/{date_str}')
        
        if req.status_code == 200:
            data = req.text
            json_str = data
            data_dict = json.loads(json_str)
            
            if current_date != start_date:
                # Отримуємо дані з API для попередньої дати
                prev_date = current_date - delta
                prev_date_str = prev_date.isoformat()
                prev_req = requests.get(f'https://russianwarship.rip/api/v1/statistics/{prev_date_str}')
                
                if prev_req.status_code == 200:
                    prev_data = prev_req.text
                    prev_json_str = prev_data
                    prev_data_dict = json.loads(prev_json_str)
                    
                    # Обчислюємо різницю між даними за поточну дату і попередню дату для всіх полів таблиці
                    diff_day = data_dict['data']['day']
                    diff_date = date_str
                    diff_personnel_units = data_dict['data']['stats']['personnel_units'] - prev_data_dict['data']['stats']['personnel_units']
                    diff_tanks = data_dict['data']['stats']['tanks'] - prev_data_dict['data']['stats']['tanks']
                    diff_armoured_fighting_vehicles = data_dict['data']['stats']['armoured_fighting_vehicles'] - prev_data_dict['data']['stats']['armoured_fighting_vehicles']
                    diff_artillery_systems = data_dict['data']['stats']['artillery_systems'] - prev_data_dict['data']['stats']['artillery_systems']
                    diff_mlrs = data_dict['data']['stats']['mlrs'] - prev_data_dict['data']['stats']['mlrs']
                    diff_aa_warfare_systems = data_dict['data']['stats']['aa_warfare_systems'] - prev_data_dict['data']['stats']['aa_warfare_systems']
                    diff_planes = data_dict['data']['stats']['planes'] - prev_data_dict['data']['stats']['planes']
                    diff_helicopters = data_dict['data']['stats']['helicopters'] - prev_data_dict['data']['stats']['helicopters']
                    diff_vehicles_fuel_tanks = data_dict['data']['stats']['vehicles_fuel_tanks'] - prev_data_dict['data']['stats']['vehicles_fuel_tanks']
                    diff_warships_cutters = data_dict['data']['stats']['warships_cutters'] - prev_data_dict['data']['stats']['warships_cutters']
                    diff_cruise_missiles = data_dict['data']['stats']['cruise_missiles'] - prev_data_dict['data']['stats']['cruise_missiles']
                    diff_uav_systems = data_dict['data']['stats']['uav_systems'] - prev_data_dict['data']['stats']['uav_systems']
                    diff_special_military_equip = data_dict['data']['stats']['special_military_equip'] - prev_data_dict['data']['stats']['special_military_equip']
                    diff_atgm_srbm_systems = data_dict['data']['stats']['atgm_srbm_systems'] - prev_data_dict['data']['stats']['atgm_srbm_systems']
                    
                    # Вставляємо різницю даних в таблицю russian_losses
                    cursor.execute('''INSERT INTO russian_losses (day, date, personnel_units, tanks, armoured_fighting_vehicles, 
                                    artillery_systems, mlrs, aa_warfare_systems, planes, helicopters, vehicles_fuel_tanks,
                                    warships_cutters, cruise_missiles, uav_systems, special_military_equip, atgm_srbm_systems)
                                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                                    (diff_day, diff_date, diff_personnel_units, diff_tanks,
                                    diff_armoured_fighting_vehicles, diff_artillery_systems, diff_mlrs,
                                    diff_aa_warfare_systems, diff_planes, diff_helicopters, diff_vehicles_fuel_tanks,
                                    diff_warships_cutters, diff_cruise_missiles, diff_uav_systems,
                                    diff_special_military_equip, diff_atgm_srbm_systems))
        
        # Зберігаємо зміни
        conn.commit()
        
    # Переходимо до наступної дати
    current_date += delta

In [11]:
cursor.execute('''
    select * from russian_losses
''').fetchall() 

[(2, '2022-02-25', 2800, 80, 516, 0, 0, 0, 10, 7, 0, 0, 0, 0, 0, 0),
 (4, '2022-02-27', 1500, 66, 190, 49, 4, 1, 17, 19, 90, 2, 0, 2, 0, 1),
 (5, '2022-02-28', 1000, 45, 110, 25, 17, 4, 2, 3, 261, 0, 0, 1, 0, -1),
 (6, '2022-03-01', 410, 7, 30, 3, 3, 2, 0, 0, 14, 0, 0, 0, 0, 0),
 (7, '2022-03-02', 130, 13, 16, 8, 16, 2, 1, 2, 50, 0, 0, 0, 0, 0),
 (8, '2022-03-03', 3160, 6, 38, 5, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0),
 (9, '2022-03-04', 166, 34, 39, 15, 8, 7, 3, 6, 49, 0, 0, 0, 0, 0),
 (10, '2022-03-05', 834, 18, 6, 0, 0, 1, 6, 3, 5, 0, 0, 0, 0, 0),
 (11, '2022-03-06', 1000, 16, 40, 4, 0, 2, 5, 8, 38, 0, 0, 1, 0, 0),
 (12, '2022-03-07', 0, 5, 14, 8, 0, 2, 2, 20, 7, 1, 0, 3, 0, 0),
 (13, '2022-03-08', 1000, 13, 37, 3, 6, 4, 2, 12, 20, 0, 0, 0, 0, 0),
 (14, '2022-03-09', 0, 14, 34, 0, 0, 1, 1, 1, 8, 0, 0, 0, 0, 0),
 (15, '2022-03-10', 0, 0, 35, 3, 0, 1, 0, 0, 44, 0, 0, 0, 0, 0),
 (16, '2022-03-11', 0, 36, 60, 2, 2, 2, 8, 2, 32, 0, 0, 0, 0, 0),
 (17, '2022-03-12', 0, 9, 40, 10, 4, 2, 1, 0, 27, 0,

In [12]:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.colheader_justify', 'center')

In [13]:
cursor.execute('''
    SELECT day, date, personnel_units, tanks, armoured_fighting_vehicles, artillery_systems, mlrs
    FROM russian_losses
''')
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

print(df)

     day     date     personnel_units  tanks  armoured_fighting_vehicles  artillery_systems  mlrs
0      2  2022-02-25       2800         80                516                     0            0 
1      4  2022-02-27       1500         66                190                    49            4 
2      5  2022-02-28       1000         45                110                    25           17 
3      6  2022-03-01        410          7                 30                     3            3 
4      7  2022-03-02        130         13                 16                     8           16 
..   ...         ...              ...    ...                         ...                ...   ...
563  566  2023-09-12        550          8                 11                    33            4 
564  567  2023-09-13        590         16                 14                    30            2 
565  568  2023-09-14        620         15                 18                    42            3 
566  569  2023-09-15

In [14]:
cursor.execute('''
    SELECT day, date, aa_warfare_systems, aa_warfare_systems  planes, helicopters, vehicles_fuel_tanks, warships_cutters 
    FROM russian_losses
''')
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

print(df)

     day     date     aa_warfare_systems  planes  helicopters  vehicles_fuel_tanks  warships_cutters
0      2  2022-02-25           0             0         7                 0                  0       
1      4  2022-02-27           1             1        19                90                  2       
2      5  2022-02-28           4             4         3               261                  0       
3      6  2022-03-01           2             2         0                14                  0       
4      7  2022-03-02           2             2         2                50                  0       
..   ...         ...                 ...     ...          ...                  ...               ...
563  566  2023-09-12           3             3         0                43                  0       
564  567  2023-09-13           2             2         0                31                  0       
565  568  2023-09-14           0             0         0                14                 

In [15]:
cursor.execute('''
    SELECT day, date, cruise_missiles, uav_systems, special_military_equip, atgm_srbm_systems
    FROM russian_losses
''')
df = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

print(df)

     day     date     cruise_missiles  uav_systems  special_military_equip  atgm_srbm_systems
0      2  2022-02-25         0              0                  0                    0        
1      4  2022-02-27         0              2                  0                    1        
2      5  2022-02-28         0              1                  0                   -1        
3      6  2022-03-01         0              0                  0                    0        
4      7  2022-03-02         0              0                  0                    0        
..   ...         ...              ...          ...                     ...                ...
563  566  2023-09-12         0             17                  4                    0        
564  567  2023-09-13         0              5                  3                    0        
565  568  2023-09-14         0             47                  5                    0        
566  569  2023-09-15         0             17               